In [1]:
pip install faker


   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 1.7 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/1.9 MB 1.5 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/1.9 MB 1.5 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/1.9 MB 1.5 MB/s eta 0:00:01
   ------------------------------------- -- 1.8/1.9 MB 1.6 MB/s eta 0:00:01
   ------------------------------------- -- 1.8/1.9 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 1.3 MB/s eta 0:00:00


In [4]:
from faker import Faker
import random
import pandas as pd
import numpy as np

In [5]:
from faker import Faker
import pandas as pd
import numpy as np
import random

fake = Faker()

# -------------------
# CONFIG
# -------------------
N_CUSTOMERS = 100_000
N_WAREHOUSES = 50
N_ORDERS = 1_000_000
N_ORDER_ITEMS = 2_000_000
N_PAYMENTS = N_ORDERS
N_CAMPAIGNS = 5000

# -------------------
# CUSTOMERS
# -------------------
customers = []
for i in range(N_CUSTOMERS):
    customers.append([
        i+1,
        fake.name(),
        fake.email(),
        fake.date_between(start_date='-2y', end_date='today'),
        fake.city(),
        random.choice(["Yes", "No"])  # Prime user
    ])
customers_df = pd.DataFrame(customers, columns=["customer_id","name","email","signup_date","location","is_prime_user"])

# Add dirtiness: missing emails, duplicate customers
customers_df.loc[customers_df.sample(frac=0.05).index, "email"] = None
customers_df = pd.concat([customers_df, customers_df.sample(frac=0.01)])  # 1% duplicates

# -------------------
# WAREHOUSES
# -------------------
warehouses = []
for i in range(N_WAREHOUSES):
    warehouses.append([
        i+1,
        fake.city(),
        round(random.uniform(0.5, 1.2), 2)  # capacity utilization
    ])
warehouses_df = pd.DataFrame(warehouses, columns=["warehouse_id","location","capacity_utilization"])

# -------------------
# ORDERS
# -------------------
orders = []
for i in range(N_ORDERS):
    orders.append([
        i+1,
        random.randint(1, N_CUSTOMERS+200),  # some invalid customer_ids
        fake.date_time_between(start_date='-1y', end_date='now'),
        random.randint(1, N_WAREHOUSES+10),  # some invalid warehouse_ids
        random.choice(["delivered","cancelled","returned"]),
        round(random.uniform(50, 1500), 2),
        random.choice([10, 12, 15, 20, 25, 500])  # delivery time, with outliers
    ])
orders_df = pd.DataFrame(orders, columns=["order_id","customer_id","order_date","warehouse_id","status","total_value","delivery_time_minutes"])

# Dirtiness: missing delivery times, mixed date formats
orders_df.loc[orders_df.sample(frac=0.05).index, "delivery_time_minutes"] = None
orders_df.loc[orders_df.sample(frac=0.01).index, "order_date"] = orders_df["order_date"].astype(str)

# -------------------
# ORDER ITEMS
# -------------------
order_items = []
for i in range(N_ORDER_ITEMS):
    order_items.append([
        i+1,
        random.randint(1, N_ORDERS),
        random.randint(1, 2000),  # product_id
        random.randint(1, 5),
        round(random.uniform(5.0, 500.0), 2)
    ])
order_items_df = pd.DataFrame(order_items, columns=["order_item_id","order_id","product_id","quantity","price_per_unit"])

# -------------------
# PAYMENTS
# -------------------
payment_methods = ["UPI", "upi", "gpay", "PhonePe", "Credit Card", "COD", "Wallet"]
payments = []
for i in range(N_PAYMENTS):
    payments.append([
        i+1,
        random.randint(1, N_ORDERS),
        fake.date_time_between(start_date='-1y', end_date='now'),
        random.choice(payment_methods),
        round(random.uniform(50.0, 1500.0), 2)
    ])
payments_df = pd.DataFrame(payments, columns=["payment_id","order_id","payment_date","method","amount"])

# Dirtiness: missing methods
payments_df.loc[payments_df.sample(frac=0.02).index, "method"] = None

# -------------------
# CAMPAIGNS
# -------------------
channels = ["App Notification", "Email", "SMS", "Push"]
campaigns = []
for i in range(N_CAMPAIGNS):
    impressions = random.randint(1000, 100000)
    clicks = random.randint(0, impressions)
    conversions = random.randint(0, clicks)
    spend = round(random.uniform(100, 10000), 2)
    campaigns.append([
        i+1,
        random.choice(channels),
        impressions,
        clicks,
        conversions,
        spend
    ])
campaigns_df = pd.DataFrame(campaigns, columns=["campaign_id","channel","impressions","clicks","conversions","spend"])

# -------------------
# SAVE TO CSV
# -------------------
customers_df.to_csv("blinkit_customers.csv", index=False)
warehouses_df.to_csv("blinkit_warehouses.csv", index=False)
orders_df.to_csv("blinkit_orders.csv", index=False)
order_items_df.to_csv("blinkit_order_items.csv", index=False)
payments_df.to_csv("blinkit_payments.csv", index=False)
campaigns_df.to_csv("blinkit_campaigns.csv", index=False)


